In [1]:
import requests
import os
import pandas as pd
import time
tokenCanvas = os.environ['tokenCanvas']

headers = {
    'Authorization': f'Bearer {tokenCanvas}',
}
params = {
    'per_page': 100,
}

def finn_rel(link_header):
    link_header_dict = {}
    for link in link_header.split(","):
        url, rel = link.strip().split(";")
        rel = rel.split('=')[1]
        link_header_dict[rel.strip().replace('"', '')] = url.strip().replace('<', '').replace('>', '')      # Kommentar: denne er med BERRE for å skape balanse i .org-fila: \"
    return link_header_dict


def sis_import(filnamn):
    url = "https://hvl.instructure.com/api/v1/accounts/1/sis_imports"
    filar = {'attachment': open(filnamn, 'rb')}
    r = requests.post(url, headers=headers, files=filar)
    resultat = r.json()
    vent = True
    forseinking = 5
    while vent:
    # Her venter eg til alt er ferdig før eg tar neste campus
        jobb_id = resultat['id']
        time.sleep(forseinking)
        url = f"https://hvl.instructure.com/api/v1/accounts/1/sis_imports/{jobb_id}"
        r = requests.get(url, headers=headers)
        resultat = r.json()
        if resultat['progress']==100:
            print("Ferdig.")
            vent = False
        else:
            print(f"Arbeider framleis, {resultat['progress']} % ferdig.")

# Avslutte manuelt innlagt i eldre emne

## Problemstilling
Når vi legg inn brukarar manuelt i emne vert dei liggande aktive etter at emneet er avslutta, for det vert ikkje sendt melding om å avslutte innmeldinga for desse, slik det vert gjort for dei som kjem frå FS.

## Løysing
Eg reknar alle emne som har `enrollment_term_id` i lista over `avslutt_semestre` som avslutta. Det er desse som får "avslutt"-melding frå FS våren 2025 (kanskje). 
Så går eg gjennom denne lista (ca. 16000 emne) og finn alle personar som framleis er aktive. 

## CD2
Eg henter først alle `courses` frå CD2, for å sjå kva eg får ut.

In [2]:
alle_emne = pd.read_csv("courses.csv")

Så må eg finne alle terminane vi har, slik at eg kan plukke ut alle emne som skal avsluttast (emne som 30.6.2025 har "2024 VÅR" som siste termin). Så kan eg jo gjere dette for andre semestre også, men det seinare.

In [ ]:
url = "https://hvl.instructure.com/api/v1/accounts/1/terms"
dr_liste = []
hentmeir = True
while hentmeir:
    print(url)
    respons = requests.get(url, headers=headers, params=params)
    if 200 <= respons.status_code < 300:
        data = respons.json()
        hentmeir = "next" in respons.headers['link']
        if hentmeir:
            url = finn_rel(respons.headers['link'])['next']
        dataliste = []
# Den neste linja varierer frå endepunkt til endepunkt
        for element in data['enrollment_terms']:
            dataliste.append(element)
        df = pd.DataFrame(dataliste, columns=['id', 'name'])
        dr_liste.append(df)
alledata = pd.concat((df for df in dr_liste if not df.empty), ignore_index=True)

avslutt = alledata[alledata['name'].str.endswith('2023 HØST')]['id']

Så kan eg leite etter emne som høyrer til i kvar termin:

In [ ]:
avslutt_emne = []
for termin in avslutt:
    avslutt_emne.append(alle_emne[alle_emne['value.enrollment_term_id'] == termin]['key.id'].to_list())
    print(f"Emne som skal avsluttast for termin {termin}: {len(alle_emne[alle_emne['value.enrollment_term_id'] == termin])}")

In [ ]:
deaktiver = []
for emnekode in avslutt_emne[5]:
    print(emnekode)
    try:
        url = f"https://hvl.instructure.com/api/v1/courses/{emnekode}/enrollments"
        innmeldingar = requests.get(url, headers=headers, params=params).json()
        for i in innmeldingar:
            if i['enrollment_state'] == 'active':
                if i['sis_user_id'] != None:
                    if i['role'] == "StudentEnrollment":
                        deaktiver.append([i['sis_user_id'],i['sis_section_id'],"Student","completed"])
                    if i['role'] == "TeacherEnrollment":
                        deaktiver.append([i['sis_user_id'],i['sis_section_id'],"Teacher","completed"])
    except:
        print(f"Feil ved {emnekode}")
    pd.DataFrame(deaktiver, columns=['user_id', 'section_id', 'role', 'status']).to_csv("deaktiver.csv", index=False)

In [ ]:
sis_import("deaktiver.csv")